# SageMaker JumpStart を用いた LightGBM (回帰)のトレーニングと推論
* JumpStart では独自のデータを用意するだけで、様々なモデルの学習と出来たモデルの推論ができる
* このノートブックでは LightGBM の回帰モデルを用いたトレーニングの動かし方を記述する
* データについては、AWS が公開しているデータを利用する
* SageMaker SDK を使ったトレーニングと推論を記載し、最後に boto3 を使った推論を記載している
* このノートブックは `Data Science 2.0` イメージ、`Python 3` カーネルで開いてください

## Tabel of Contents
* [事前準備](#事前準備)
  * [モジュールのインポート](#モジュールのインポート)
  * [データ取得](#データ取得)
* [SageMaker JumpStart を使って CUI(SageMaker SDK) でトレーニングと推論](#SageMaker-JumpStart-を使って-CUI(SageMaker-SDK)-でトレーニングと推論)
  * [トレーニング](#トレーニング)
    * [データアップロード](#データアップロード)
    * [トレーニングパラメータの取得](#トレーニングパラメータの取得)
    * [トレーニングジョブ実行](#トレーニングジョブ実行)
  * [推論](#推論)
    * [推論パラメータの取得](#トレーニングパラメータの取得)
    * [推論エンドポイント作成](#推論エンドポイント作成)
* [boto3 で推論](#boto3-で推論)
  * [定数やクライアントの設定](#定数やクライアントの設定)
  * [モデルと推論コードを tar.gz に固める](#モデルと推論コードを-tar.gz-に固める)
  * [boto3 でSageMaker でモデルの作成](#boto3-でSageMaker-でモデルの作成)
  * [boto3 でエンドポイントの設定を作成](#boto3-でエンドポイントの設定を作成)
  * [boto3 でエンドポイントを作成する](#boto3-でエンドポイントを作成する)
  * [boto3 で推論する](#boto3-で推論する)
  * [boto3 でエンドポイント他を削除](#boto3-でエンドポイント他を削除)


## 事前準備
### モジュールのインポート

In [ ]:
import sagemaker
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.estimator import Estimator
from sagemaker.session import Session
from sagemaker import hyperparameters
import json
import pandas as pd
from typing import Final

### データ取得
公開されている回帰用データを使う

In [ ]:
data_dir: Final[str] = 'regression_data'
!if [ -d ./{data_dir} ]; then rm -rf ./{data_dir}/;fi
!mkdir ./{data_dir}/
!aws s3 sync  s3://jumpstart-cache-prod-us-east-1/training-datasets/tabular_regress/ ./{data_dir}/

## SageMaker JumpStart を使って CUI(SageMaker SDK) でトレーニングと推論
### トレーニング

#### データアップロード

* トレーニングデータについて
    * JumpStart で自分のデータでトレーニングするには予め S3 に配置する(トレーニング実行時に S3 の URI を指定する)
* データの持ち方について
    * csv 形式でファイル名を data.csv にする必要がある(トレーニングコードが data.csv しか受け付けないようになっている)
    * 訓練用データの `train/data.csv` は必ず用意する
    * 評価用データの`validation/data.csv` はオプション
    * テスト用データの `test/data.csv` はトレーニング時にもちろん使わないがまとめてアップロードしているので副次的にアップロードされる
    * ターゲット変数は必ず 0 列目に入れること(トレーニングコードが 0 列目をターゲット変数として認識するように実装されている)
* カテゴリー変数について
    * データディレクトリのルートに任意の json ファイルを１つだけ含むことでカテゴリカル変数を扱うことができる
    * カテゴリー変数は、0 以上の整数(Int32の範囲内)でエンコードされている必要がある
    * カテゴリー変数は列のインデックスで辞書形式でキーに `cat_index_list` で、値に列のインデックスをリスト形式で渡す
    * 今回は 1 列目がカテゴリー変数

データの確認(JumpStart を動かすのには不要)

In [ ]:
# !cat {data_dir}/cat_index.json
# pd.read_csv(f'{data_dir}/train/data.csv',header=None).head()

* データアップロードは [upload_data](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session.upload_data) メソッドを利用して、ディレクトリまるごと S3 にアップロードする
* ここでは SageMaker のデフォルトバケット(`sagemaker-{region}-{account}`にアップロードしているが、任意のバケットを選択するときは `bucket` 引数を使用する
* ここで出力される URI は、GUI で入力する値でもある（GUI の場合は、S3 の URI を入力したあと `Train` をクリックすれば学習が開始される  

<div align="left">
<img src="./image/smj_gui_regression.png" alt="写真" width="436" height="420">
</div>

In [ ]:
# 使うデータを S3 にアップロード
input_s3_uri: Final[str] = sagemaker.session.Session().upload_data(
    f'./{data_dir}/',
    key_prefix = 'sagemaker-jumpstart/lightgbm_regression/data'
)
print(f'アップロード先 : \n{input_s3_uri}')

#### トレーニングパラメータの取得
* JumpStart は予めコンテナやトレーニングコードを用意しているので、そのパラメータを取得する

##### 定数の設定

In [ ]:
# JumpStart で動かすモデルとバージョン、インスタンスタイプと台数を設定
model_id: Final[str] = 'lightgbm-regression-model'
model_version: Final[str] = '*'
training_instance_type: Final[str] = 'ml.m5.xlarge'
instance_count: Final[int] = 1

##### ロール名を取得
トレーニングジョブを動かす際に、トレーニングインスタンスに割り当てるロールを取得

In [ ]:
# JumpStart で動かすトレーニングジョブにアタッチするロールを取得(Notebook と同一)
role: Final[str] = sagemaker.get_execution_role()
print(role)

##### Fine-Tune の元となるモデルの URI を取得
* JumpStart は Fine-Tune が基本なので、Fine-Tune の元となるモデルの URI を取得
* ただし、LightGBM は Fine-Tune するものではないので regression するという設定値だけが格納されている
* [sagemaker.model_uris.retrieve](https://sagemaker.readthedocs.io/en/stable/api/utility/model_uris.html#sagemaker.model_uris.retrieve) メソッドで取得できる

In [ ]:
base_model_uri: Final[str] = model_uris.retrieve(model_id=model_id, model_version=model_version, model_scope="training")
print(f'モデルの URI:\n{base_model_uri}')

設定を確認したい場合は下記を実行( JumpStart を動かすのには不要な作業)

In [ ]:
# model_dir = 'train-lightgbm-regression-model'
# !aws s3 cp {base_model_uri} ./
# !if [ -d ./{model_dir} ]; then rm -rf {model_dir}/;fi
# !mkdir {model_dir}/
# !tar zxvf train-lightgbm-regression-model.tar.gz -C ./{model_dir}/
# !cat {model_dir}/train-pytorch-lightgbm-lightgbmregress.json

##### トレーニングコードの S3 URI を取得
* トレーニングコードは AWS が管理する S3 に格納されており、トレーニングジョブを定義する時に使うため取得する  
* [sagemaker.script_uris.retrieve](https://sagemaker.readthedocs.io/en/stable/api/utility/script_uris.html#sagemaker.script_uris.retrieve) メソッドで取得できる

In [ ]:
training_script_uri: Final[str] = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
print(f'コードの URI:\n{training_script_uri}')

* トレーニングコードを確認したい場合は下記を実行( JumpStart を動かすのには不要な作業)
* トレーニングコードをカスタマイズしたい場合はダウンロードして編集する

In [ ]:
# training_script_dir: Final[str] = 'lightgbm_regression_training_script_dir'
# !aws s3 cp {training_script_uri} ./
# !if [ -d ./{training_script_dir} ]; then rm -rf ./{training_script_dir}/;fi
# !mkdir ./{training_script_dir}/
# !tar zxvf sourcedir.tar.gz -C ./{training_script_dir}/
# !pygmentize ./{training_script_dir}/transfer_learning.py

##### トレーニングコンテナイメージの URI を取得
* AWS が管理する ECR に格納されており、その URI を取得する
* [sagemaker.image_uris.retrieve](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html#sagemaker.image_uris.retrieve) メソッドで取得できる

In [ ]:
training_image_uri: Final[str] = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="training",
    model_id=model_id,
    model_version=model_version,
    instance_type=training_instance_type,
)
print(f'コンテナの URI:\n{training_image_uri}')

##### デフォルトのハイパーパラメータを取得
* [sagemaker.hyperparameters.retrieve_default](https://sagemaker.readthedocs.io/en/stable/api/utility/hyperparameters.html#sagemaker.hyperparameters.retrieve_default) メソッドで取得できる
* ハイパーパラメータを変える場合は取得結果の辞書を上書きする

In [ ]:
hps = hyperparameters.retrieve_default(
    model_id=model_id,
    model_version=model_version,
)
print(f'ハイパーパラメータ\n{json.dumps(hps,indent=4)}')

#### トレーニングジョブ実行
* 通常の SageMaker Training と同じ様に [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator) クラスから `estimator` インスタンスを生成し、 [fit](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator.fit) メソッドで実行する
* 今まで取得してきた設定値を引数に入れて `estimator` インスタンスを生成する
* `training_script_uri` について、ローカルで書き換えた場合はローカルのディレクトリを指定する
* fit の引数にトレーニングデータの S3 URI を指定する

In [ ]:
estimator = Estimator(
    image_uri=training_image_uri,
    source_dir=training_script_uri,
    model_uri=base_model_uri,
    entry_point="transfer_learning.py",
    role=role,
    hyperparameters=hps,
    instance_count=instance_count,
    instance_type=training_instance_type,
)
estimator.fit({"training": input_s3_uri})


### 推論

#### 推論パラメータの取得
* JumpStart は予めコンテナや推論コードを用意しているので、そのパラメータを取得する

##### トレーニングコードの S3 URI を取得
* 推論コードは AWS が管理する S3 に格納されており、モデルデプロイに使うため取得する  
* [sagemaker.script_uris.retrieve](https://sagemaker.readthedocs.io/en/stable/api/utility/script_uris.html#sagemaker.script_uris.retrieve) メソッドで取得できる

In [ ]:
inference_script_uri: Final[str] = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)
print(f'推論コードのURL:\n{inference_script_uri}')

* 推論コードを確認したい場合は下記を実行( JumpStart を動かすのには不要な作業)
* 推論コードをカスタマイズしたい場合はダウンロードして編集する

In [ ]:
# inference_script_dir: Final[str] = 'lightgbm_regresion_inference_script_dir'
# !aws s3 cp {inference_script_uri} ./
# !if [ -d ./{inference_script_dir} ]; then rm -rf ./{inference_script_dir}/;fi
# !mkdir ./{inference_script_dir}/
# !tar zxvf sourcedir.tar.gz -C ./{inference_script_dir}/
# !pygmentize ./{inference_script_dir}/inference.py

##### 推論コンテナイメージの URI を取得
* AWS が管理する ECR に格納されており、その URI を取得する
* [sagemaker.image_uris.retrieve](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html#sagemaker.image_uris.retrieve) メソッドで取得できる

In [ ]:
inference_image_uri: Final[str] = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=training_instance_type,
)
print(f'コンテナの URI:\n{inference_image_uri}')

#### 推論エンドポイント作成
[Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator) の [deploy](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy) メソッドでエンドポイント作成を行う

In [ ]:
predictor = estimator.deploy(
    instance_type = 'ml.m5.large',
    initial_instance_count  = 1,
    entry_point='inference.py',
    source_dir=inference_script_uri,
    image_uri = inference_image_uri
    
)

#### 推論実行
* エンドポイントはデフォルトだと `text/csv` しか受け付けないので(推論コードの `inference.py` と `constants.py` を参照)、呼び出しもと(predictor)側に [CSVSerializer](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html#sagemaker.serializers.CSVSerializer) を設定する
* `CSVSerializer` を設定すると、API へのリクエスト([predict](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor.predict))時に `content_type='text/csv'` が設定され、また ndarray を渡しても裏側で csv にシリアライズしてくれる

In [ ]:
# csvに変換して、csv 形式でリクエストを投げてくれるようになる
predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# csv でリクエストするパターン
json.loads(predictor.predict(pd.read_csv(f'{data_dir}/test/data.csv',header=None).iloc[:,1:].to_csv(header=False,index=False)).decode('utf-8'))['prediction']
# # ndarray でリクエストするパターン
# json.loads(predictor.predict(pd.read_csv(f'{data_dir}/test/data.csv',header=None).iloc[:,1:].values).decode('utf-8'))['prediction']

#### エンドポイント削除
* エンドポイントを削除することでインスタンスが停止される
* [delete_endpoint](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor.delete_endpoint) で削除できる

In [ ]:
predictor.delete_endpoint()

## boto3 で推論
エンドポイント作成や推論は SageMaker SDK ではなく、boto3 からやることも多いのでやり方を紹介

### 定数やクライアントの設定

In [ ]:
import boto3
sm_client = boto3.client('sagemaker')
smr_client = boto3.client('sagemaker-runtime')
endpoint_inservice_waiter = sm_client.get_waiter('endpoint_in_service')

In [ ]:
model_name: Final[str] = 'LightgbmRegressor'
endpoint_config_name: Final[str] = model_name + 'EndpointConfig'
endpoint_name: Final[str] = model_name + 'Endpoint'

### モデルと推論コードを tar.gz に固める
推論エンドポイントを立ち上げるためには、SageMaker 上でモデルを登録する必要がある。  
ここでいう`モデル`とは、「機械学習モデル+推論コード」を tar.gz の S3 URI と、モデルを動かすコンテナなどを指す。  
トレーニングが終わった段階では、lightgbm の学習済モデル(pkl) だけなので、当然推論コードを含まないので、  
推論コードを梱包して S3 にアップロードしなおす(SageMaker SDK だと裏側で勝手にやってくれていた)。  
  
推論コードは、`tar.gz` のルートディレクトリに `code` ディレクトリを配置しその直下に`inference.py`で置くと勝手に読んでくれる。(名前を変えることもできるか環境変数をいじる必要が出てくるのでお勧めしない）

In [ ]:
# トレーニングの記録からモデルの URI を取得して、ローカルにダウンロードする
!aws s3 cp {estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']} ./
# 先程使った 推論コードをダウンロードする
!aws s3 cp {inference_script_uri} ./

# モデルを解凍
inference_model_dir: Final[str] = 'model'
!if [ -d ./{inference_model_dir} ]; then rm -rf ./{inference_model_dir}/;fi
!mkdir ./{inference_model_dir}/
!tar zxvf ./model.tar.gz -C ./{inference_model_dir}/

# コードを追加
inference_code_dir: Final[str] = 'code'
!if [ -d ./{inference_code_dir} ]; then rm -rf ./{inference_code_dir}/;fi
!mkdir ./{inference_code_dir}/
!tar zxvf ./sourcedir.tar.gz -C ./{inference_code_dir}/
!mv ./code/ model/

# 再圧縮
!rm ./{inference_model_dir}.tar.gz
%cd {inference_model_dir}/
!tar zcvf model.tar.gz .
%cd ..

# モデルとトレーニングコードの tar.gz を S3 にアップロード
inference_model_uri: Final[str] = sagemaker.session.Session().upload_data(
    f'./{inference_model_dir}/{inference_model_dir}.tar.gz',
    key_prefix = 'sagemaker-jumpstart/lightgbm/model'
)
print(f'アップロード先 : \n{inference_model_uri}')

### boto3 で SageMaker でモデルの作成
アップロードしたモデル `model.tar.gz` と、コンテナイメージを設定する  
[create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model) メソッドで設定する

In [ ]:
response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        # SageMaker SDK の時と同じ URI を指定
        'Image': inference_image_uri,
        # SageMaker SDK の時と同じ URI を指定
        'ModelDataUrl': inference_model_uri,
    },
    # SageMaker SDK の時と同じ role を指定
    ExecutionRoleArn=role,
)
print(response)

### boto3 でエンドポイントの設定を作成
使用するモデル、インスタンスの種類と台数などを設定する。  
[create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config) メソッドで設定する

In [ ]:
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.xlarge',
        },
    ],
)
print(response)

### boto3 でエンドポイントを作成する
[create_endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint) メソッドで作成する

In [ ]:
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
# エンドポイントが立ち上がるまで待つ
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)

### boto3 で推論する
[invoke_endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint)で推論を実行できる。  
client は `boto3.client('sagemaker')` ではなく、`boto3.client('sagemaker-runtime')`なことに注意。

In [ ]:
request_args = {
    'EndpointName': endpoint_name,
    'ContentType' : 'text/csv',
    'Body' : pd.read_csv(f'{data_dir}/test/data.csv',header=None).iloc[:,1:].to_csv(header=False, index=False)
}
response = smr_client.invoke_endpoint(**request_args)
json.loads(response['Body'].read())['prediction']

### boto3 でエンドポイント他を削除

In [ ]:
r = sm_client.delete_endpoint(EndpointName=endpoint_name)
r = sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
r = sm_client.delete_model(ModelName=model_name)